In [1]:
import json
import random
from helper import set_openai_key, test_openai_api, create_openai_client, print_pretty

In [2]:
set_openai_key()

test_openai_api()

client = create_openai_client()

API key set successfully.
Hi there! How can I assist you today?


## Function Calling

![img](https://cdn.openai.com/API/docs/images/function-calling-diagram-steps.png)

### Get weather

In [3]:
messages = [{"role": "user", "content": "What is the weather like in Boston?"}]

functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location"]
        }
    }
]

response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    temperature=0,
    functions=functions,
    function_call="auto"  # Ensures function calling is automatically handled
)

print(response)


ChatCompletion(id='chatcmpl-B4lPrR9UrAfSD4TPr2xPu6l1c3JUt', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA","unit":"fahrenheit"}', name='get_current_weather'), tool_calls=None))], created=1740475279, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier='default', system_fingerprint='fp_bf9cb2c77f', usage=CompletionUsage(completion_tokens=23, prompt_tokens=86, total_tokens=109, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [4]:
print_pretty(response.dict())

{
    "choices": [
        {
            "finish_reason": "function_call",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": null,
                "function_call": {
                    "arguments": "{\"location\":\"Boston, MA\",\"unit\":\"fahrenheit\"}",
                    "name": "get_current_weather"
                },
                "refusal": null,
                "role": "assistant",
                "tool_calls": null
            }
        }
    ],
    "created": 1740475279,
    "id": "chatcmpl-B4lPrR9UrAfSD4TPr2xPu6l1c3JUt",
    "model": "gpt-4-turbo-2024-04-09",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": "fp_bf9cb2c77f",
    "usage": {
        "completion_tokens": 23,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_predict

In [5]:
# Let define function in local
def get_current_weather(location, unit="fahrenheit"):
    """Simulates fetching current weather for a given location."""
    conditions = ["Sunny", "Cloudy", "Rainy", "Stormy", "Snowy"]
    temperature = random.randint(-10, 100) if unit == "fahrenheit" else round((random.randint(-10, 100) - 32) * 5/9, 1)
    
    return {
        "location": location,
        "temperature": str(temperature),
        "unit": unit,
        "condition": random.choice(conditions)
    }

In [6]:
assistant_message = response.choices[0].message.dict()

if assistant_message.get("function_call"):

    function_name = assistant_message["function_call"]["name"]

    print("name:",function_name)
    
    function_args = json.loads(assistant_message["function_call"]["arguments"])

    print("arguments:",function_args)

    if function_name == "get_current_weather":
        
        function_response = get_current_weather(**function_args)

        messages.append({"role": "function", "name": function_name, "content": json.dumps(function_response)})
        
        final_response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            temperature=0
        )
        
        print_pretty(final_response.dict())
    

name: get_current_weather
arguments: {'location': 'Boston, MA', 'unit': 'fahrenheit'}
{
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": "The current weather in Boston, MA is rainy with a temperature of -1\u00b0F.",
                "function_call": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": null
            }
        }
    ],
    "created": 1740475286,
    "id": "chatcmpl-B4lPyCdbcNsHayy7hPOTy4NJ6NlHQ",
    "model": "gpt-4-turbo-2024-04-09",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": "fp_bf9cb2c77f",
    "usage": {
        "completion_tokens": 18,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_t

In [7]:
print(final_response.choices[0].message.content)

The current weather in Boston, MA is rainy with a temperature of -1°F.


### Send email

**Tool choice**

By default the model will determine when and how many tools to use. You can force specific behavior with the tool_choice parameter.

- Auto: (Default) Call zero, one, or multiple functions. `tool_choice: "auto"`

- Required: Call one or more functions. `tool_choice: "required"`

- Forced Function: Call exactly one specific function. `tool_choice: {"type": "function", "function": {"name": "get_weather"}}`


In [8]:
messages=[{"role": "user", "content": "Can you send an email to ilan@example.com and katia@example.com saying hi?"}]

In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": "Send an email to a given recipient with a subject and message.",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {
                        "type": "string",
                        "description": "The recipient email address."
                    },
                    "subject": {
                        "type": "string",
                        "description": "Email subject line."
                    },
                    "body": {
                        "type": "string",
                        "description": "Body of the email message."
                    }
                },
                "required": [
                    "to",
                    "subject",
                    "body"
                ],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

In [10]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

print_pretty(response.dict())

{
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": null,
                "function_call": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "function": {
                            "arguments": "{\"to\": \"ilan@example.com\", \"subject\": \"Hello!\", \"body\": \"Hi Ilan,\\n\\nJust wanted to say hi!\\n\\nBest regards,\"}",
                            "name": "send_email"
                        },
                        "id": "call_Rhw2bfEEYRBGFKp8FR9lZk6j",
                        "type": "function"
                    },
                    {
                        "function": {
                            "arguments": "{\"to\": \"katia@example.com\", \"subject\": \"Hello!\", \"body\": \"Hi Katia,\\n\\nJust wanted to say hi!

In [11]:
def send_email(name, args):
    """
    Mock function to simulate sending an email.
    Prints the email details for testing purposes.
    """
    print(f"Function: {name}")
    print(f"Sending email to: {args['to']}")
    print(f"Subject: {args['subject']}")
    print(f"Body: {args['body']}")
    
    # Simulate success response
    return json.dumps({"status": "success", "message": "Email sent successfully."})

In [12]:
for tool_call in response.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    print(name,args)
    result = send_email(name, args)

    # Append assistant's tool call response
    messages.append({
        "role": "assistant",
        "content": None,
        "tool_calls": [
            {
                "id": tool_call.id,
                "type": "function",
                "function": {
                    "name": name,
                    "arguments": json.dumps(args)
                }
            }
        ]
    })

    # # Append tool response correctly
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": result
    })

send_email {'to': 'ilan@example.com', 'subject': 'Hello!', 'body': 'Hi Ilan,\n\nJust wanted to say hi!\n\nBest regards,'}
Function: send_email
Sending email to: ilan@example.com
Subject: Hello!
Body: Hi Ilan,

Just wanted to say hi!

Best regards,
send_email {'to': 'katia@example.com', 'subject': 'Hello!', 'body': 'Hi Katia,\n\nJust wanted to say hi!\n\nBest regards,'}
Function: send_email
Sending email to: katia@example.com
Subject: Hello!
Body: Hi Katia,

Just wanted to say hi!

Best regards,


In [13]:
messages

[{'role': 'user',
  'content': 'Can you send an email to ilan@example.com and katia@example.com saying hi?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_Rhw2bfEEYRBGFKp8FR9lZk6j',
    'type': 'function',
    'function': {'name': 'send_email',
     'arguments': '{"to": "ilan@example.com", "subject": "Hello!", "body": "Hi Ilan,\\n\\nJust wanted to say hi!\\n\\nBest regards,"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_Rhw2bfEEYRBGFKp8FR9lZk6j',
  'content': '{"status": "success", "message": "Email sent successfully."}'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_to4S05VzWKSScBqHgYMSHb84',
    'type': 'function',
    'function': {'name': 'send_email',
     'arguments': '{"to": "katia@example.com", "subject": "Hello!", "body": "Hi Katia,\\n\\nJust wanted to say hi!\\n\\nBest regards,"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_to4S05VzWKSScBqHgYMSHb84',
  'content': '{"status": "success", "message": "Email sent successfu

In [14]:
final_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

print_pretty(final_response.dict())

{
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": "I've sent the emails to both Ilan and Katia saying hi! If there's anything else you need, feel free to ask.",
                "function_call": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": null
            }
        }
    ],
    "created": 1740475302,
    "id": "chatcmpl-B4lQEO3alY6HbEspRMKfkOnbwFCqZ",
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": "fp_eb9dce56a8",
    "usage": {
        "completion_tokens": 28,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens": 220,
        

### Search knowledge base


In [29]:
tools = [
    {
        "type": "function",  # Specifies that this tool is a function.
        "function": {
            "name": "search_knowledge_base",  # Function name to be called by the AI model.
            "description": "Query a knowledge base to retrieve relevant info on a topic.",  # Describes what this function does.
            "parameters": {  # Defines the expected input parameters for this function.
                "type": "object",  # The parameters are structured as a JSON object.
                "properties": {  # Specifies the individual parameters.
                    "query": {  
                        "type": "string",  # The query must be a string.
                        "description": "The user question or search query."  # Explanation of what the query represents.
                    },
                    "options": {  # Additional options for refining the search.
                        "type": "object",  # This is a nested object containing search options.
                        "properties": {  # Specifies the sub-options within "options".
                            "num_results": {
                                "type": "number",  # Must be a number.
                                "description": "Number of top results to return."  # Defines how many search results to retrieve.
                            },
                            "domain_filter": {
                                "type": [
                                    "string",
                                    "null"
                                ],  # Can be a string (specific domain) or null (no filter).
                                "description": "Optional domain to narrow the search (e.g. 'finance', 'medical'). Pass null if not needed."  
                                # Allows filtering by domain, e.g., only returning results from "finance" or "medical".
                            },
                            "sort_by": {
                                "type": [
                                    "string",
                                    "null"
                                ],  # Can be a string (sort option) or null (no sorting).
                                "enum": [
                                    "relevance",  # Sort by relevance to the query.
                                    "date",  # Sort by the most recent results.
                                    "popularity",  # Sort by most popular results.
                                    "alphabetical"  # Sort alphabetically.
                                ],
                                "description": "How to sort results. Pass null if not needed."  
                                # Specifies sorting criteria for the retrieved results.
                            }
                        },
                        "required": [
                            "num_results",
                            "domain_filter",
                            "sort_by"
                        ],  # These options must always be provided.
                        "additionalProperties": False  # Prevents extra, undefined properties from being included.
                    }
                },
                "required": [
                    "query",
                    "options"
                ],  # Both "query" and "options" must be present.
                "additionalProperties": False  # No other properties are allowed outside the defined ones.
            },
            "strict": True  # Enforces strict validation on parameters.
        }
    }
]


In [30]:
messages=[{"role": "user", "content": "Can you find information about ChatGPT in the AI knowledge base?"}]

In [31]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

print_pretty(response.dict())

{
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "audio": null,
                "content": null,
                "function_call": null,
                "refusal": null,
                "role": "assistant",
                "tool_calls": [
                    {
                        "function": {
                            "arguments": "{\"query\":\"ChatGPT\",\"options\":{\"num_results\":5,\"domain_filter\":\"AI\",\"sort_by\":\"relevance\"}}",
                            "name": "search_knowledge_base"
                        },
                        "id": "call_MNaX2XBWyajYwlfdeqJaU2FH",
                        "type": "function"
                    }
                ]
            }
        }
    ],
    "created": 1740476386,
    "id": "chatcmpl-B4lhiumdu9JniupRyurYTAC7indVq",
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "service_tier": "de

In [36]:
def search_knowledge_base(name, args):
    """
    Mock function to simulate querying a knowledge base.
    """
    query = args.get("query")
    options = args.get("options", {})
    num_results = options.get("num_results", 3)
    domain_filter = options.get("domain_filter", None)
    sort_by = options.get("sort_by", "relevance")

    # Mock response data
    results = [
        {"title": "Introduction to ChatGPT", "snippet": "ChatGPT is an AI model developed by OpenAI..."},
        {"title": "ChatGPT Use Cases", "snippet": "ChatGPT can be used for customer support, coding, and more..."},
        {"title": "How ChatGPT Works", "snippet": "ChatGPT uses a transformer-based neural network to generate responses..."},
        {"title": "ChatGPT The Best AI model", "snippet": "Will AI take over the world? ChatGPT is becoming more intelligent."}
    ]
    
    # Apply mock filters
    if domain_filter:
        results = [r for r in results if domain_filter.lower() in r["title"].lower()]

    # Apply sorting (mock behavior)
    if sort_by == "alphabetical":
        results = sorted(results, key=lambda x: x["title"])
    elif sort_by == "date":
        results = results[::-1]  # Reverse list as a placeholder for recent-first sorting

    # Limit the number of results
    results = results[:num_results]

    # Format response
    response = {"query": query, "results": results}
    
    return json.dumps(response, indent=2)

# Example test call
test_args = {
    "query": "ChatGPT",
    "options": {
        "num_results": 2,
        "domain_filter": None,
        "sort_by": "relevance"
    }
}

print(search_knowledge_base("search_knowledge_base", test_args))

{
  "query": "ChatGPT",
  "results": [
    {
      "title": "Introduction to ChatGPT",
      "snippet": "ChatGPT is an AI model developed by OpenAI..."
    },
    {
      "title": "ChatGPT Use Cases",
      "snippet": "ChatGPT can be used for customer support, coding, and more..."
    }
  ]
}


In [37]:
for tool_call in response.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    print(name,args)
    result = search_knowledge_base(name, args)

    # Append assistant's tool call response
    messages.append({
        "role": "assistant",
        "content": None,
        "tool_calls": [
            {
                "id": tool_call.id,
                "type": "function",
                "function": {
                    "name": name,
                    "arguments": json.dumps(args)
                }
            }
        ]
    })

    # # Append tool response correctly
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": result
    })

search_knowledge_base {'query': 'ChatGPT', 'options': {'num_results': 5, 'domain_filter': 'AI', 'sort_by': 'relevance'}}


In [38]:
messages

[{'role': 'user',
  'content': 'Can you find information about ChatGPT in the AI knowledge base?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_MNaX2XBWyajYwlfdeqJaU2FH',
    'type': 'function',
    'function': {'name': 'search_knowledge_base',
     'arguments': '{"query": "ChatGPT", "options": {"num_results": 5, "domain_filter": "AI", "sort_by": "relevance"}}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_MNaX2XBWyajYwlfdeqJaU2FH',
  'content': '{\n  "query": "ChatGPT",\n  "results": []\n}'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_MNaX2XBWyajYwlfdeqJaU2FH',
    'type': 'function',
    'function': {'name': 'search_knowledge_base',
     'arguments': '{"query": "ChatGPT", "options": {"num_results": 5, "domain_filter": "AI", "sort_by": "relevance"}}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_MNaX2XBWyajYwlfdeqJaU2FH',
  'content': '{\n  "query": "ChatGPT",\n  "results": [\n    {\n      "title": "ChatGPT The Best AI model",

In [39]:
final_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

print(final_response.choices[0].message.content)

Here's the most relevant information I found about ChatGPT in the AI knowledge base:

- **ChatGPT The Best AI model**: Highlights include ChatGPT becoming more intelligent and discussions about whether AI will take over the world.

If you need further details or additional topics, feel free to ask!
